In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
# reading the table 
df_coffee_review = pd.read_csv("./web_scaping/output_data_csv/coffee_bean_cleaned.csv")
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,['Jersey City'],Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250.0,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,[' Taipei'],Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


# Basic Cleaning
---

In [4]:
# number of rows before drop_duplicates
df_coffee_review["coffee name"].count()

1626

In [5]:
# drop dulipcates from the same roaster name and coffee name
df_coffee_review = df_coffee_review.drop_duplicates(subset= ["coffee name", "roaster name", "coffee origin country"], keep= "first")

In [6]:
# number of rows after drop_duplicates
df_coffee_review["coffee name"].count()

1568


### removing list format in roaster city

In [7]:
# remove list format in roaster city
roaster_city = []
for cell in df_coffee_review["roaster city"]:
    cell = cell[2:-2].strip() # removing the ([') in the front and (']) at the end
    cell = re.sub("', '.", ";", cell) # finding [', 'anything] and replacing into [;]
    roaster_city.append(cell)

df_coffee_review["roaster city"] = roaster_city
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64.0,82.0,30.00,250.0,March 2023,9.0,9.0,9.0,9.0,9.0,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60.0,77.0,11.52,226.8,March 2023,9.0,9.0,9.0,9.0,8.0,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


In [8]:
# formating all the columns
df_coffee_review[["whole bean (agtron)", "after grinding (agtron)", "aroma", "acidity", "body", "flavor", "aftertaste"]] =\
      df_coffee_review[["whole bean (agtron)", "after grinding (agtron)", "aroma", "acidity", "body", "flavor", "aftertaste"]].fillna(0)
df_coffee_review = df_coffee_review.astype({"whole bean (agtron)" : int, "after grinding (agtron)" : int, "aroma" : int,
                                             "acidity" : int, "body" : int, "flavor" : int, "aftertaste" : int})
df_coffee_review.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (paragraph)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64,82,30.00,250.0,March 2023,9,9,9,9,9,"Wildly tropical, fruity and deep. Passion frui...",Produced by Wilton Benitez entirely of the Pin...
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60,77,11.52,226.8,March 2023,9,9,9,9,8,"Complex, nuanced, multi-layered. Black currant...",Produced by the Kariruki family from trees of ...


## Identifying the coffee plant species in the species (paragraph)
---

In [9]:
# drop all rows that does not have any species information
df_coffee_review = df_coffee_review.dropna(subset= ["species (paragraph)"])

In [10]:
# check is there any more NaN values in the species columns
np.where(df_coffee_review["species (paragraph)"].values == np.NaN)

(array([], dtype=int64),)

In [11]:
# all arabica species name
coffee_species = ["Java", "Geisha", "Sudan Rume", "Bourbon", "Typica", "Caturra", "Villa Sarchi", "Pacas", "Venecia", "Tekisic", "SL28", 
                  "K7", "KP423", "Jackson 2/1257", "Bourbon Mayaguez 71", "Bourbon Mayaguez 139", "Mibirizi", "Maragogipe", "Nyasaland",
                  "Pache", "Harrar Rwanda", "Harar", "Harraghe", "POP3303/21", "SL14", "SL34", "Mundo Novo", "Catuai", "Pacamara", "Catimor", "T8667", 
                  "T5175", "Lempira", "Costa Rica 95", "Catisic", "IHCAFE 90", "Oro Azteca", "Fronton", "Anacafe 14", "Sarchimor T5296", 
                  "Cuscatleco", "Parainema", "Limani", "Obata Rojo", "IAPAR 59", "Marsellesa", "Batian", "RAB C15", "S795", "Ruiru 11", 
                  "Centroamericano", "Nayarita", "Evaluna", "Mundo Maya", "Milenio", "Starmaya", "Casiopea", "H3", "Castillo", "Heirloom",
                  "Wush Wush", "wushwush", str(74110), str(74158), "Gesha", "Landrace", "Robusta", "Mocca","Moka", "Jember"]

In [12]:
# for loop for searching the arabica species name used in that coffee bean
species_list = []
for index, items in df_coffee_review.iterrows():
    name = items["coffee name"]
    species_para = items["species (paragraph)"]
    row = []
    for species in coffee_species:
        # if the coffee species name is in the paragraph or in the coffee name, then put the species name into row list
        if species.lower() in species_para.lower() or species.lower() in name.lower():
            row.append(species)
    # replace Gesha to Geisha and Jember to S795 and Harraghe to Harrar Rwanda (they are the same species)
    row = list(map(lambda x: x.replace("Gesha", "Geisha"), row))
    row = list(map(lambda x: x.replace("Jember", "S795"), row))
    row = list(map(lambda x: x.replace("Harar", "Harrar Rwanda"), row))
    row = list(map(lambda x: x.replace("Harraghe", "Harrar Rwanda"), row))
    # drop all the dup in the list
    row = [*set(row)] # set() -> return a collection of unique objects
    row_str = ";".join(row)
    species_list.append(row_str)

In [13]:
# removing species (paragraph) column
df_coffee_review = df_coffee_review.drop(columns= "species (paragraph)")
# adding a column for the arabica species
df_coffee_review["species (variety)"] = species_list

## Seperating the coffee bean dataframe into sub-dataframe
---
- Tables: coffee_main, origin_country, roast_level, roaster, species_full and species_linked
- create IDs: coffee_ID, country_ID, roaster_ID, species_ID, city_ID

In [14]:
# copy df_coffee_review into full_df_coffee
full_df_coffee = df_coffee_review.copy()
full_df_coffee.head(2)

,coffee name,coffee rating,roaster name,roaster country/state,roaster city,coffee origin country,coffee origin city,roast level,whole bean (agtron),after grinding (agtron),price,pricing unit (g),review date,aroma,acidity,body,flavor,aftertaste,blind ass (paragraph),species (variety)
0,Colombia Pink Bourbon,95,modcup,New Jersey,Jersey City,Colombia,Piendamó;Cauca Department,Light,64,82,30.00,250.0,March 2023,9,9,9,9,9,"Wildly tropical, fruity and deep. Passion frui...",Bourbon
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taiwan,Taipei,Kenya,Kiambu County,Medium-Light,60,77,11.52,226.8,March 2023,9,9,9,9,8,"Complex, nuanced, multi-layered. Black currant...",SL28;SL34


In [15]:
full_df_coffee.columns

Index(['coffee name', 'coffee rating', 'roaster name', 'roaster country/state',
       'roaster city', 'coffee origin country', 'coffee origin city',
       'roast level', 'whole bean (agtron)', 'after grinding (agtron)',
       'price', 'pricing unit (g)', 'review date', 'aroma', 'acidity', 'body',
       'flavor', 'aftertaste', 'blind ass (paragraph)', 'species (variety)'],
      dtype='object')

In [16]:
# create coffee ID (1 to 1680)
coffee_np = np.arange(1, 1568)

# adding coffee_ID column into df_coffee_review
full_df_coffee["coffee_ID"] = ["C" + str(x) for x in coffee_np]

### Origin Location Table

In [17]:
# origin table
origin_df = full_df_coffee.loc[:, ["coffee_ID", "coffee origin country"]]
origin_df.head(2)

,coffee_ID,coffee origin country
0,C1,Colombia
1,C2,Kenya


In [18]:
# spliting origin country into list
origin_df["coffee origin country"] = [n.split(";") for n in origin_df["coffee origin country"]]
# expanding the table by using explode
origin_df_expand = origin_df.explode("coffee origin country")
origin_df_expand["coffee_ID"].count()

1679

In [19]:
# creating unqiue ID for origin table
origin_np = np.arange(1, 1680)

# adding origin_id column into origin_df
origin_df_expand["origin_id"] = ["O" + str(x) for x in origin_np]
origin_df_expand.head(2)

,coffee_ID,coffee origin country,origin_id
0,C1,Colombia,O1
1,C2,Kenya,O2


In [20]:
# creating a list to combine all the country in origin_df and roaster_df
country_list = list(origin_df_expand["coffee origin country"].unique())

### City table

In [21]:
# city table
city_df= full_df_coffee.loc[:, ["roaster city"]].groupby("roaster city").first().reset_index()
# spliting muliple cities and putting into a list
city_df["roaster city"] = [n.split(";") for n in city_df["roaster city"]]
# expanding the table
city_df = city_df.explode("roaster city")
city_df["roaster city"].count()

212

In [22]:
# generate number from 1 to 213
city_np = np.arange(1, 213)

# adding coffee_ID column into df_coffee_review
city_df["city_ID"] = ["City" + str(x) for x in city_np]
city_df.head(2)

,roaster city,city_ID
0,,City1
1,Acton,City2


In [23]:
city_df_cleaned = city_df.rename(columns= {"roaster city" : "roaster_city"})
city_df_cleaned.head(2)

,roaster_city,city_ID
0,,City1
1,Acton,City2


#### Finding country according to city name

In [24]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(timeout = 3, user_agent="PDS") # create a geolocator object

roaster_country_list = [] # create a list to store country name

for city in city_df["roaster city"]:
    try:
        location = geolocator.geocode(city, language= "en") # get the location data for the city
        country = location.address.split(",")[-1].strip() # select the country name from the output
        roaster_country_list.append(country)
        print(f"{city} Done")
    except:
        roaster_country_list.append("")

# add the country list into dataframe
city_df["roaster_country"] = roaster_country_list

Acton Done
Albuquerque Done
Anchorage Done
Annapolis Done
Anoka Done
Antigua Done
Arcata Done
Arlington Done
Ashland Done
Atlanta Done
Austin Done
Bedford Done
Bellingham Done
Belmar Done
Berkeley Done
Berlin Done
Beverly Done
Big Lake Done
Billings Done
Bogota Done
Boston Done
Boulder Done
Bozeman Done
Branford Done
Brooklyn Done
Buffalo Done
Burlington Done
Calgary Done
Alberta Done
Cambria Done
Captain Cook Done
Carbondale Done
Carlsbad Done
Changhua Done
Changhua County Done
Charlotte Done
Charlottesville Done
Chia-Yi Done
Chicago Done
Chino Done
Chiya-Yi Done
Columbus Done
Crystal Lake Done
Daejeon Done
Denver Done
Des Moines Done
Douliou City Done
Driftless Done
Duluth Done
Durango Done
Durham Done
Emeryville Done
Eugene Done
Everett Done
Floyd Done
Fort Bragg Done
Fort Lauderdale Done
Glendale Done
Hamilton Done
Harrisonburg Done
Henderson Done
Highland Park Done
Hillsborough Done
Hilo Done
Holualoa Done
Hong Kong Done
Houston Done
Hsinchu Done
Hsinchu City Done
Hudson Done
Itha

In [25]:
city_df.head(2)

,roaster city,city_ID,roaster_country
0,,City1,
1,Acton,City2,United Kingdom


### Country Table

In [26]:
# counting the length of country list
print(f"Before adding roaster country: {len(country_list)}")

Before adding roaster country: 36


In [27]:
# adding roaster country that is not in the country list
for country in city_df["roaster_country"]:
    if country not in country_list:
        country_list.append(country)

print(f"After adding roaster country: {len(country_list)}")

After adding roaster country: 47


In [28]:
# generate ID for country table
country_np = np.arange(1, 48)
country_ids = ["ctry" + str(x) for x in country_np]
len(country_ids)

47

In [29]:
# create table for country name (columns: country name and country ID)
country_df = pd.DataFrame({"country_id" : country_ids,
                           "country_name" : country_list})
country_df.head(4)

,country_id,country_name
0,ctry1,Colombia
1,ctry2,Kenya
2,ctry3,Costa Rica
3,ctry4,Ethiopia


### Roaster table

In [30]:
# roaster table
roaster_grp = full_df_coffee.loc[:, ["roaster name", "roaster city"]].groupby("roaster name")
roaster_df = pd.concat([roaster_grp["roaster city"].first()], axis= "columns", sort= False).reset_index()
roaster_df["roaster name"].count()

332

In [31]:
# generate number from 1 to 333
roaster_np = np.arange(1, 333)

# adding coffee_ID column into df_coffee_review
roaster_df["roaster_ID"] = ["R" + str(x) for x in roaster_np]
roaster_df.head(2)

,roaster name,roaster city,roaster_ID
0,94 Fresh Coffee,Kaohsiung,R1
1,A.R.C.,Hong Kong,R2


In [32]:
# remove roaster city column
roaster_df_drop = roaster_df.drop(columns= "roaster city")
# rename the roaster name column
roaster_df_cleaned = roaster_df_drop.rename(columns= {"roaster name" : "roaster_name"})
roaster_df_cleaned.head(2)

,roaster_name,roaster_ID
0,94 Fresh Coffee,R1
1,A.R.C.,R2


#### Roaster city linked table

In [33]:
roaster_link_df_merge = roaster_df.merge(city_df_cleaned, how= "left", left_on = "roaster city", right_on= "roaster_city")\
    .drop(columns= ["roaster city", "roaster_city", "roaster name"])
roaster_link_df_merge["roaster_ID"].count()

346

In [34]:
# generate number from 1 to 333
roaster_city_np = np.arange(1, 347)

# adding coffee_ID column into df_coffee_review
roaster_link_df_merge["roaster_city_ID"] = ["RS" + str(x) for x in roaster_city_np]
roaster_link_df_merge.head(2)

,roaster_ID,city_ID,roaster_city_ID
0,R1,City81,RS1
1,R2,City68,RS2


In [35]:
# re-organise the table columns
roaster_link_df_cleaned = roaster_link_df_merge[["roaster_city_ID", "roaster_ID", "city_ID"]]
roaster_link_df_cleaned.head(2)

,roaster_city_ID,roaster_ID,city_ID
0,RS1,R1,City81
1,RS2,R2,City68


#### merging origin_df and city_df with country_df

In [36]:
# merging origin_df with the country ID and remove dup column
origin_df_merge = origin_df_expand.merge(country_df, how= "left", left_on = "coffee origin country", right_on= "country_name")\
    .drop(columns = ["coffee origin country", "country_name"])
# rename country_id column into coffee origin country ID
origin_df_cleaned = origin_df_merge.rename(columns= {"country_id" : "coffee_origin_country_id"})
origin_df_cleaned.head(2)

,coffee_ID,origin_id,coffee_origin_country_id
0,C1,O1,ctry1
1,C2,O2,ctry2


In [37]:
# merging city_df with the country ID and remove dup column
city_df_merge = city_df.merge(country_df, how="left", left_on= "roaster_country", right_on= "country_name")\
            .drop(columns= ["roaster_country", "country_name"])
# rename columns 
city_df_cleaned = city_df_merge.rename(columns= {"roaster city" : "roaster_city"})
city_df_cleaned.head(2)

,roaster_city,city_ID,country_id
0,,City1,ctry37
1,Acton,City2,ctry38


### Roast level table

In [38]:
roast_lv_df = full_df_coffee.loc[:,["roast level"]].groupby("roast level").first()
# there is only five different roast level -> np.range(1, 6)
roast_lv_ids = ["RL" + str(x) for x in np.arange(1, 6)]

# adding the ids into roast_lv_ids
roast_lv_df["roast_lv_id"] = roast_lv_ids
roast_lv_df = roast_lv_df.reset_index()

# rename columns
roast_lv_df_cleaned = roast_lv_df.rename(columns= {"roast level" : "roast_level"})
roast_lv_df_cleaned
 

,roast_level,roast_lv_id
0,Dark,RL1
1,Light,RL2
2,Medium,RL3
3,Medium-Dark,RL4
4,Medium-Light,RL5


### Coffee main table

In [39]:
# getting the columns needed in main table
main_df = full_df_coffee.loc[:,["coffee_ID", "coffee name", "review date", "coffee rating", "roast level", "roaster name",
                                "whole bean (agtron)", "after grinding (agtron)", "price", "pricing unit (g)", "aroma", 
                                "acidity", "body", "flavor", "aftertaste"]]
main_df.head(2)

,coffee_ID,coffee name,review date,coffee rating,roast level,roaster name,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste
0,C1,Colombia Pink Bourbon,March 2023,95,Light,modcup,64,82,30.00,250.0,9,9,9,9,9
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,March 2023,94,Medium-Light,Buon Caffe,60,77,11.52,226.8,9,9,9,9,8


In [40]:
# change the reivew date into datatime object
main_df["review date"] = pd.to_datetime(main_df["review date"], format= "%B %Y")
main_df.head(2)

,coffee_ID,coffee name,review date,coffee rating,roast level,roaster name,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste
0,C1,Colombia Pink Bourbon,2023-03-01,95,Light,modcup,64,82,30.00,250.0,9,9,9,9,9
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,2023-03-01,94,Medium-Light,Buon Caffe,60,77,11.52,226.8,9,9,9,9,8


In [41]:
# merging roast level df with main_df
main_df_ = main_df.merge(roast_lv_df_cleaned, how= "left", left_on = "roast level", right_on= "roast_level")
# merging roaster_df with main_df
main_df_merge=  main_df_.merge(roaster_df_cleaned, how= "left", left_on = "roaster name", right_on= "roaster_name")
main_df_merge.head(2)

,coffee_ID,coffee name,review date,coffee rating,roast level,roaster name,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste,roast_level,roast_lv_id,roaster_name,roaster_ID
0,C1,Colombia Pink Bourbon,2023-03-01,95,Light,modcup,64,82,30.00,250.0,9,9,9,9,9,Light,RL2,modcup,R331
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,2023-03-01,94,Medium-Light,Buon Caffe,60,77,11.52,226.8,9,9,9,9,8,Medium-Light,RL5,Buon Caffe,R38


In [42]:
# caluate the price per gram
main_df_merge["price_per_kg_USD"] = main_df_merge["price"]/main_df_merge["pricing unit (g)"]*1000
main_df_merge.head(2)


,coffee_ID,coffee name,review date,coffee rating,roast level,roaster name,whole bean (agtron),after grinding (agtron),price,pricing unit (g),aroma,acidity,body,flavor,aftertaste,roast_level,roast_lv_id,roaster_name,roaster_ID,price_per_kg_USD
0,C1,Colombia Pink Bourbon,2023-03-01,95,Light,modcup,64,82,30.00,250.0,9,9,9,9,9,Light,RL2,modcup,R331,120.000000
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,2023-03-01,94,Medium-Light,Buon Caffe,60,77,11.52,226.8,9,9,9,9,8,Medium-Light,RL5,Buon Caffe,R38,50.793651


In [43]:
# dropping the unwanted columns
main_df_drop = main_df_merge.drop(columns= ["roaster name", "roaster_name", "roast level", "roast_level", "price", "pricing unit (g)"])
main_df_drop.head(2)

,coffee_ID,coffee name,review date,coffee rating,whole bean (agtron),after grinding (agtron),aroma,acidity,body,flavor,aftertaste,roast_lv_id,roaster_ID,price_per_kg_USD
0,C1,Colombia Pink Bourbon,2023-03-01,95,64,82,9,9,9,9,9,RL2,R331,120.000000
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,2023-03-01,94,60,77,9,9,9,9,8,RL5,R38,50.793651


In [44]:
# rename columns name
main_df_cleaned = main_df_drop.rename(columns= {"coffee name" : "coffee_name", "review date" : "review_date",
                                                 "coffee rating" : "coffee_rating", "whole bean (agtron)" : "agtron_WB",
                                                 "after grinding (agtron)" : "agtron_AG"})
main_df_cleaned.head(2)

,coffee_ID,coffee_name,review_date,coffee_rating,agtron_WB,agtron_AG,aroma,acidity,body,flavor,aftertaste,roast_lv_id,roaster_ID,price_per_kg_USD
0,C1,Colombia Pink Bourbon,2023-03-01,95,64,82,9,9,9,9,9,RL2,R331,120.000000
1,C2,Kenya Kiambu Mandela Estate AA Washed Process,2023-03-01,94,60,77,9,9,9,9,8,RL5,R38,50.793651


In [45]:
roaster_df_cleaned.head(2)

,roaster_name,roaster_ID
0,94 Fresh Coffee,R1
1,A.R.C.,R2


In [46]:
city_df_cleaned.head(2)

,roaster_city,city_ID,country_id
0,,City1,ctry37
1,Acton,City2,ctry38


### Species link table

In [47]:
# species_link table
species_df = full_df_coffee.loc[:,["coffee_ID", "species (variety)"]]
# spliting the species into list and expanding the whole table
species_df["species (variety)"] = [n.split(";") for n in species_df["species (variety)"]]
species_df = species_df.explode("species (variety)")
species_df

,coffee_ID,species (variety)
0,C1,Bourbon
1,C2,SL28
1,C2,SL34
2,C3,Geisha
3,C4,
...,...,...
1621,C1563,
1622,C1564,
1623,C1565,Geisha
1624,C1566,


In [48]:
# creating unqiue ID for species_link
species_coffee_np = np.arange(1, 2060)

# adding species_coffee_id column into species_link
species_df["species_coffee_id"] = ["CS" + str(x) for x in species_coffee_np]
species_df.head(2)

,coffee_ID,species (variety),species_coffee_id
0,C1,Bourbon,CS1
1,C2,SL28,CS2


### Species table

In [49]:
# creating the ids for species table
species_ids = ["S" + str(x) for x in np.arange(1, 60)]
len(species_ids)

59

In [59]:
# reading species csv
species_full_df = pd.read_csv("./web_scaping/output_data_csv/coffee_plant_species_cleaned.csv")
species_full_df["species_id"] = species_ids
species_full_df.head(2)

,species name,brief intro,plant stature,leaf tip colour,bean size,optial altitude low (5N_5S),optial altitude id (5-15N_5-15S),optial altitude high (15N_15S),quality potential at high altitude,yield potential,disease: coffee leaf rust,disease: coffee berry disease,disease: nematodes,nutrition requirment,planting density,species_id
0,Anacafe 14,"Very high yielding variety, with rust resistan...",Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,4000-5000 a/ha (using single-stem pruning),S1
1,Batian,"A tall variety that combines high yields, tole...",Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,2000-3000 a/ha (using multiple-stem pruning),S2


In [60]:
# drop all unwanted columns
species_full_df_drop = species_full_df.drop(columns= ["brief intro", "planting density"])
species_full_df_drop.head(2)

,species name,plant stature,leaf tip colour,bean size,optial altitude low (5N_5S),optial altitude id (5-15N_5-15S),optial altitude high (15N_15S),quality potential at high altitude,yield potential,disease: coffee leaf rust,disease: coffee berry disease,disease: nematodes,nutrition requirment,species_id
0,Anacafe 14,Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,S1
1,Batian,Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,S2


In [61]:
# rename columns name
species_full_cleaned = species_full_df_drop.rename(columns= {"species name" : "species_name", "plant stature" : "plant_stature",
                                                             "leaf tip colour" : "leaf_colour", "bean size" : "bean_size",
                                                             "optial altitude low (5N_5S)" : "altitude_low_lat",
                                                             "optial altitude id (5-15N_5-15S)" : "altitude_medium_lat",
                                                             "optial altitude high (15N_15S)" : "altitude_high_lat",
                                                             "quality potential at high altitude" : "quality_at_high_altitude",
                                                             "yield potential" : "yield_potential", "disease: coffee leaf rust" : "CLR_resistant",
                                                             "disease: coffee berry disease" : "CBD_resistant",
                                                             "disease: nematodes" : "nematodes_resistant", "nutrition requirment" : "nutrition_require"})

species_full_cleaned.head(2)

,species_name,plant_stature,leaf_colour,bean_size,altitude_low_lat,altitude_medium_lat,altitude_high_lat,quality_at_high_altitude,yield_potential,CLR_resistant,CBD_resistant,nematodes_resistant,nutrition_require,species_id
0,Anacafe 14,Dwarf/Compact,Green,Very Large,1200,900,700,Good,High,Resistant,Susceptible,Susceptible,High,S1
1,Batian,Tall,Green or Bronze,Very Large,1000,700,400,Very Good,High,Tolerant,Resistant,Susceptible,Medium,S2


In [62]:
species_full_cleaned["nutrition_require"].str.len().max()

14.0

In [63]:
# merging the species table with the linking table
species_df_merge = species_df.merge(species_full_df[["species_id", "species name"]], how= "left", left_on= "species (variety)", right_on= "species name")
species_df_cleaned = species_df_merge.drop(columns= ["species (variety)", "species name"])
species_df_cleaned.head(2)


,coffee_ID,species_coffee_id,species_id
0,C1,CS1,S3
1,C2,CS2,S46


#### Saving all the sub tables

In [64]:
origin_df_cleaned.to_csv("./cleaned_csv/origin_country.csv", index= False)
roast_lv_df_cleaned.to_csv("./cleaned_csv/roast_lv.csv", index= False)
roaster_df_cleaned.to_csv("./cleaned_csv/roaster.csv", index= False)
species_full_cleaned.to_csv("./cleaned_csv/species_detail.csv", index= False)
species_df_cleaned.to_csv("./cleaned_csv/species_link.csv", index= False)
main_df_cleaned.to_csv("./cleaned_csv/coffee_main.csv", index= False)
country_df.to_csv("./cleaned_csv/country.csv", index= False)
city_df_cleaned.to_csv("./cleaned_csv/city.csv", index= False)
roaster_link_df_cleaned.to_csv("./cleaned_csv/roaster_city.csv", index= False)